In [ ]:
from __future__ import annotations

# Compatibility with latest Atomic Agents (>=1.1)
from atomic_agents.agents.base_agent import BaseAgent as BaseChatAgent
from atomic_agents.agents.base_agent import BaseAgentConfig as BaseChatAgentConfig
from atomic_agents.lib.base.base_io_schema import BaseIOSchema

# Minimal Instructor client (no actual LLM calls) to satisfy BaseAgentConfig
from instructor.client import Instructor
from opik.integrations.atomic_agents import OpikContextProvider, track_atomic_agents

from pydantic import Field

# Atomic Agents × Opik – Quick-Start 🚀

Welcome!  This notebook-style script shows **step-by-step** how to instrument an
Atomic Agents pipeline with Opik for full tracing and observability.

👀 **What you’ll learn**
1. Install the required packages
2. Enable automatic tracing in **one line**
3. Build a tiny `EchoAgent`
4. Inspect the trace in the Opik UI

> Run it interactively (VS Code / Jupyter) or as a plain `python` script – the
> trace is created either way.

## 1. Installation
```bash
pip install "opik[atomic_agents]" atomic-agents instructor
```

### Set environment variables

```bash
export OPIK_API_KEY="<your-api-key>"
# Optional: point to self-hosted instance
# export OPIK_BASE_URL="http://localhost:3000"
```

In [ ]:
track_atomic_agents(project_name="atomic-quickstart")

In [ ]:
class ChatMessage(BaseIOSchema):
    """A simple chat message for input and output"""

    chat_message: str = Field(..., description="User message")


class EchoAgent(BaseChatAgent):
    """Agent that echoes back the input message."""

    input_schema = ChatMessage
    output_schema = ChatMessage

    # No need to override run() - BaseAgent handles the workflow
    # The LLM client will be called automatically with the proper schemas

In [ ]:
# Create dummy client that returns an echo response
dummy_instructor_client = Instructor(
    client=None,
    create=lambda **kwargs: ChatMessage(chat_message="Echo: Hello Atomic Agents!"),
)

agent = EchoAgent(
    config=BaseChatAgentConfig(model="gpt-3.5-turbo", client=dummy_instructor_client),
)

result = agent.run(ChatMessage(chat_message="Hello Atomic Agents!"))
print("Agent result:", result)

In [ ]:
provider = OpikContextProvider(
    project_name="atomic-quickstart", trace_id="<replace-with-id>"
)
print("\nProvider renders:\n", provider.get_info())